In [1]:
%pip install -qU langchain
%pip install pypdf
%pip install chromadb
%pip install openai
%pip install tiktoken
%pip install nltk
# !pip install --upgrade nltk
# !pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
import chromadb

from langchain.document_loaders import PyPDFLoader 
from langchain.vectorstores import Chroma 
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings 
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
os.environ["OPENAI_API_KEY"] = 'Kindly User Your own API'
llm = OpenAI(openai_api_key="OPENAI_API_KEY")
from sentence_transformers import SentenceTransformer


In [3]:
from langchain.document_loaders import PyPDFLoader
load=[]
loader = PyPDFLoader("D:\Research Internship\LangChain Practise\Final Output\Business Project.pdf")

In [4]:
load.append(loader)

In [5]:
file2=PyPDFLoader("D:\Research Internship\LangChain Practise\Final Output\MIS.pdf")
load.append(file2)
# docs = loader.load_and_split()
# print(len(docs))
# print(docs[2])

pages = load[0].load_and_split()
for i in range(len(load)):
    if i == 0 : continue
    pages += load[i].load_and_split()
    print(i)
    # pages.append(page)


1


In [42]:
print(pages[3].page_content)

Research
Methodology
(Data
Collection
Method,
Sampling
procedure,
Questionnaire)
D a t a
C o l l e c t i o n
M e t h o d :
W e
used
a
mixed-methods
approach
for
data
collection,
which
involved
both
primary
and
secondary
sources.
W e
conducted
physical
visits
to
the
electronics
shop
to
understand
their
business
model
and
operations.
W e
also
reviewed
relevant
literature,
including
industry
reports
and
academic
papers,
to
gain
a
better
understanding
of
the
challenges
facing
the
electronics
retail
industry .
S a m p l i n g
P r o c e d u r e :
W e
used
a
purposive
sampling
technique
to
select
the
participants
for
our
study .
W e
included
employees
and
customers
of
the
electronics
shop
in
our
study .
W e
selected
employees
from
dif ferent
departments,
including
sales,
marketing,
and
inventory
management,
to
gain
a
holistic
understanding
of
the
business.
W e
selected
customers
based
on
their
frequency
of
visits
and
purchases.
Q u e s t i o n n a i r e :
W e
developed
a
Google
form
of
the
qu

In [14]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [23]:
print(type(pages[0].metadata))

<class 'dict'>


In [35]:
#Our sentences we like to encode
# sentences = ['This framework generates embeddings for each input sentence',
#     'Sentences are passed as a list of string.',
#     'The quick brown fox jumps over the lazy dog.']
sentence=''
metadata={}
id=[]
for i in range(len(pages)):
    sentence=sentence+""+pages[i].page_content
    metadata.update(pages[i].metadata)
    id.append("id "+str(i))
#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

In [36]:
print(embeddings)

[[-0.01371735 -0.04285153 -0.01562862 ...  0.10017826  0.12365723
  -0.0422967 ]
 [ 0.05645249  0.05500239  0.03137961 ...  0.06650875  0.08491522
  -0.0332849 ]
 [ 0.04393352  0.05893442  0.04817837 ...  0.05216279  0.05610652
   0.10206389]]


In [39]:
print(type(embeddings))
client = chromadb.Client()
collection = client.create_collection(
    # documents=pages;
    embeddings=embeddings,
    metadatas=metadata,
    ids=id
)
# collection.add(
#     embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2], ...
#     metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
#     ids=["id1", "id2", "id3", ...]
# )

Using embedded DuckDB without persistence: data will be transient


<class 'numpy.ndarray'>


TypeError: LocalAPI.create_collection() got an unexpected keyword argument 'embeddings'

In [17]:
vectordb = Chroma.from_documents(pages, embedding = embeddings, persist_directory=".")

Using embedded DuckDB with persistence: data will be stored in: .


AttributeError: 'numpy.ndarray' object has no attribute 'embed_documents'

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
pdf_qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.9) , vectordb.as_retriever(), memory=memory)

In [ ]:
query = "What is Research problem statmenet"
result = pdf_qa({"question": query})
print("Answer:")
result["answer"]

In [ ]:
query2 = "How is Questionnaire develped"
result = pdf_qa({"question": query2})
print("Answer:")
result["answer"]

In [ ]:
query3 = "What is information as a resource"
result = pdf_qa({"question": query3})
print("Answer:")
result["answer"]